In [75]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, roc_auc_score, accuracy_score
from sklearn.inspection import permutation_importance

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


In [62]:
vic_df_clean=pd.read_csv('data/victim_data_with_clean_ori9code.csv')
vic_df_clean=vic_df_clean.drop(['Unnamed: 0'],axis=1)
vic_df_clean=vic_df_clean.drop(np.where(vic_df_clean['race'].isnull())[0],axis=0)
vic_df_clean

,id,name,date,armed_wapo,age,gender,race,city_wapo,state,signs_of_mental_illness,...,race_N,race_O,race_W,geo_Rural,geo_Suburban,geo_Undetermined,geo_Urban,threat_attack,threat_other,threat_undetermined
0,3,Tim Elliot,2015-01-02,gun,53.0,M,A,Shelton,WA,True,...,0,0,0,1,0,0,0,1,0,0
1,4,Lewis Lee Lembke,2015-01-02,gun,47.0,M,W,Aloha,OR,False,...,0,0,1,0,1,0,0,1,0,0
2,5,John Paul Quintero,2015-01-03,unarmed,23.0,M,H,Wichita,KS,False,...,0,0,0,0,0,0,1,0,1,0
3,8,Matthew Hoffman,2015-01-04,toy weapon,32.0,M,W,San Francisco,CA,True,...,0,0,1,0,0,0,1,1,0,0
4,9,Michael Rodriguez,2015-01-04,nail gun,39.0,M,H,Evans,CO,False,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5427,6254,Brandon Evans,2020-10-24,knife,33.0,M,W,Lawrenceburg,IN,False,...,0,0,1,0,1,0,0,1,0,0
5428,6255,Jakerion Shmond Jackson,2020-10-24,gun,19.0,M,B,Sylvester,GA,False,...,0,0,0,0,0,0,0,1,0,0
5430,6266,Walter Wallace,2020-10-26,knife,27.0,M,B,Philadelphia,PA,True,...,0,0,0,0,0,0,0,0,1,0
5432,6268,Isaac Lemoine Christensen,2020-10-27,gun,38.0,M,W,Herriman,UT,False,...,0,0,1,0,1,0,0,0,1,0


In [65]:
np.sum(vic_df_clean.isnull())

id                           0
name                        33
date                         0
armed_wapo                 199
age                         90
gender                       0
race                         0
city_wapo                    0
state                        0
signs_of_mental_illness      0
threat_level                 0
flee                       222
body_camera                  0
longitude                  153
latitude                   153
is_geocoding_exact           0
wapo_id                      0
street_address              46
city_mpv                     1
state_code                   0
zipcode                     23
county                       9
police_department            0
ORI_agency_id                0
ORI9                         0
ORI9_match_type              0
is_male                      0
is_tasered_and_shot          0
is_armed_gun                 0
is_armed_knife               0
is_unarmed                   0
is_armed_other               0
race_A  

In [15]:
vic_df_clean.dtypes

id                           int64
name                        object
date                        object
armed_wapo                  object
age                        float64
gender                      object
race                        object
city_wapo                   object
state                       object
signs_of_mental_illness       bool
threat_level                object
flee                        object
body_camera                   bool
longitude                  float64
latitude                   float64
is_geocoding_exact            bool
wapo_id                      int64
street_address              object
city_mpv                    object
state_code                  object
zipcode                    float64
county                      object
police_department           object
ORI_agency_id               object
ORI9                        object
ORI9_match_type              int64
is_male                       bool
is_tasered_and_shot           bool
is_armed_gun        

In [21]:
diff_city_ind=np.where(vic_df_clean['city_wapo']!=vic_df_clean['city_mpv'])

In [26]:
vic_df_clean.iloc[diff_city_ind][['name','city_wapo','state','city_mpv','state_code','police_department']]

,name,city_wapo,state,city_mpv,state_code,police_department
27,Michael Goebel,Franklin County,MO,Robertsville,MO,"Louis County Police Department, Jefferson Coun..."
35,Nathan Massey,Evangeline Parish,LA,Ville Platte,LA,Louisiana State Police
45,Miguel Angel de Santos-Rodriguez,Chapeno,TX,Roma,TX,U.S. Border Patrol
50,Demaris Turner,Fort Lauderdale,FL,Lauderhill,FL,Lauderhill Police Department
71,Victor Manuel Reyes,Houston,TX,Katy,TX,Houston Police Department
...,...,...,...,...,...,...
5388,Crystal Renee Starling-McClinton,Hilliard,FL,Hillard,FL,Nassau County Sheriff's Office
5390,Tyran Dent,New York,NY,Queens,NY,New York Police Department
5392,Sylvia Veronica Kirchner,Horry County,SC,Myrtle Beach,SC,Horry County Police Department
5393,Anthony Michael Legato,Pine County,MN,Hinkley,MN,Pine County Sheriff's Office


In [30]:
vic_df_clean.iloc[np.where(vic_df_clean['state']!=vic_df_clean['state_code'])]\
[['name','id','wapo_id','city_wapo','state','city_mpv','state_code','police_department']]

,name,id,wapo_id,city_wapo,state,city_mpv,state_code,police_department
271,Isaac Jimenez,356,356,Alton,IL,Affton,MO,Affton Police Department
1640,Ricardo Tenorio,1874,1874,Memphis,TN,West Memphis,AR,Shelby County Sheriff's Office
2824,Zoe Dowdell,3213,3213,New Britain,CT,McMinnville,TN,McMinnville Police Department
2825,Clifford Keller,3212,3212,McMinnville,TN,Lexington,VA,Rockbridge County Sheriff's Office
2826,Robert Allen Morrison,3214,3214,Lexington,VA,New Britain,CT,New Britain Police Department
4026,Thomas Johnson,4586,4586,Bastrop,LA,Kittitas,WA,Kittitas County Sheriff's Office
4027,Juan Manuel Flores Del Toro,4590,4590,Kittitas,WA,Tampa,FL,Hillsborough County Sheriff's Office
4028,Jesus Calderon,4596,4596,Tampa,FL,University City,MO,University City Police Department
4092,Brian Dellaann Butts,4655,4655,Portland,OR,Kalama,WA,Cowlitz County Sheriff's Office
4106,James Freeman,4654,4654,Watauga,TX,Shelby,MT,"Toole County Sheriff's Office, Great Falls Pol..."


In [24]:
vic_df_lemas=pd.read_csv('data/victim_data_lemas_basic.csv')
vic_df_lemas=vic_df_lemas.drop(['Unnamed: 0'])

,Unnamed: 0,id,name,date,armed_wapo,age,gender,race,city_wapo,state,...,PERS_AMIND,FRAC_AMIND,PERS_ASIAN,FRAC_ASIAN,PERS_HAWPI,FRAC_HAWPI,PERS_MULTI,FRAC_MULTI,PERS_UNK,FRAC_UNK
0,0,3,Tim Elliot,2015-01-02,gun,53.0,M,A,Shelton,WA,...,3,0.055556,0,0.000000,0,0.000000,0,0.000000,0,0.000000
1,1,4,Lewis Lee Lembke,2015-01-02,gun,47.0,M,W,Aloha,OR,...,3,0.007463,16,0.039801,0,0.000000,0,0.000000,3,0.007463
2,2,5,John Paul Quintero,2015-01-03,unarmed,23.0,M,H,Wichita,KS,...,2,0.002950,21,0.030973,0,0.000000,0,0.000000,0,0.000000
3,3,8,Matthew Hoffman,2015-01-04,toy weapon,32.0,M,W,San Francisco,CA,...,7,0.002971,385,0.163413,140,0.059423,0,0.000000,38,0.016129
4,4,9,Michael Rodriguez,2015-01-04,nail gun,39.0,M,H,Evans,CO,...,0,0.000000,0,0.000000,0,0.000000,0,0.000000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5650,5650,6266,Walter Wallace,2020-10-26,knife,27.0,M,B,Philadelphia,PA,...,6,0.000995,98,0.016249,11,0.001824,0,0.000000,0,0.000000
5651,5651,6270,Michael K. Nelson,2020-10-26,gun,27.0,M,NaN,Alexandria,VA,...,1,0.003205,18,0.057692,0,0.000000,3,0.009615,0,0.000000
5652,5652,6268,Isaac Lemoine Christensen,2020-10-27,gun,38.0,M,W,Herriman,UT,...,0,0.000000,1,0.008929,0,0.000000,0,0.000000,0,0.000000
5653,5653,6269,Clifton Gorman Spencer,2020-10-27,gun,35.0,M,NaN,Chattanooga,TN,...,1,0.002004,7,0.014028,0,0.000000,0,0.000000,0,0.000000


In [33]:
# traits of police department
vic_df_lemas.columns

Index(['Unnamed: 0', 'id', 'name', 'date', 'armed_wapo', 'age', 'gender',
       'race', 'city_wapo', 'state', 'signs_of_mental_illness', 'threat_level',
       'flee', 'body_camera', 'longitude', 'latitude', 'is_geocoding_exact',
       'wapo_id', 'street_address', 'city_mpv', 'state_code', 'zipcode',
       'county', 'police_department', 'ORI_agency_id', 'ORI9',
       'ORI9_match_type', 'is_male', 'is_tasered_and_shot', 'is_armed_gun',
       'is_armed_knife', 'is_unarmed', 'is_armed_other', 'race_A', 'race_B',
       'race_H', 'race_N', 'race_O', 'race_W', 'geo_Rural', 'geo_Suburban',
       'geo_Undetermined', 'geo_Urban', 'threat_attack', 'threat_other',
       'threat_undetermined', 'AGENCYNAME', 'CITY', 'ZIPCODE', 'STATE',
       'COUNTY', 'FIPS', 'POPSERVED', 'POPGROUP', 'AGENCYTYPE', 'PERS_TOTAL',
       'PERS_WHITE', 'FRAC_WHITE', 'PERS_BLACK', 'FRAC_BLACK', 'PERS_HISP',
       'FRAC_HISP', 'PERS_AMIND', 'FRAC_AMIND', 'PERS_ASIAN', 'FRAC_ASIAN',
       'PERS_HAWPI', 'FRAC_HA

In [77]:
predictors=['race','body_camera','PERS_TOTAL','FRAC_WHITE','FRAC_BLACK','FRAC_HISP','FRAC_AMIND','FRAC_ASIAN','FRAC_HAWPI','FRAC_MULTI','FRAC_UNK']
temp=vic_df_lemas[predictors].copy()
temp=temp.dropna()
temp

# train test split
X_train,X_test,y_train,y_test=train_test_split(temp.drop(['race'],axis=1),temp.race,random_state=42,train_size=0.8)

In [80]:
X_train

,city_wapo,state,body_camera,police_department,PERS_TOTAL,FRAC_WHITE,FRAC_BLACK,FRAC_HISP,FRAC_AMIND,FRAC_ASIAN,FRAC_HAWPI,FRAC_MULTI,FRAC_UNK
2282,Fresno,CA,True,Fresno Police Department,740,0.517568,0.056757,0.351351,0.008108,0.044595,0.021622,0.000000,0.000000
2025,Gary,IN,False,Lake County Sheriff's Office,157,0.662420,0.146497,0.191083,0.000000,0.000000,0.000000,0.000000,0.000000
2800,Lodi,CA,False,Lodi Police Department,409,0.616137,0.029340,0.268949,0.002445,0.078240,0.000000,0.004890,0.000000
973,Aurora,CO,False,Aurora Police Department,685,0.837956,0.042336,0.078832,0.007299,0.013139,0.001460,0.018978,0.000000
1253,Splendora,TX,False,Montgomery County Sheriff's Office,-18,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4790,Tahlequah,OK,False,Cherokee Nation Marshal Service,20,0.750000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000
473,Lancaster,CA,False,Los Angeles County Sheriff's Department,9351,0.391295,0.089937,0.446049,0.002567,0.070153,0.000000,0.000000,0.000000
3263,Las Vegas,NV,True,Las Vegas Metropolitan Police Department,3387,0.687629,0.098022,0.130204,0.006200,0.045763,0.008562,0.023620,0.000000
4031,Flagstaff,AZ,False,Arizona Department of Public Safety,1144,0.793706,0.021853,0.166084,0.009615,0.008741,0.000000,0.000000,0.000000


In [34]:
vic_df_clean.columns

Index(['id', 'name', 'date', 'armed_wapo', 'age', 'gender', 'race',
       'city_wapo', 'state', 'signs_of_mental_illness', 'threat_level', 'flee',
       'body_camera', 'longitude', 'latitude', 'is_geocoding_exact', 'wapo_id',
       'street_address', 'city_mpv', 'state_code', 'zipcode', 'county',
       'police_department', 'ORI_agency_id', 'ORI9', 'ORI9_match_type',
       'is_male', 'is_tasered_and_shot', 'is_armed_gun', 'is_armed_knife',
       'is_unarmed', 'is_armed_other', 'race_A', 'race_B', 'race_H', 'race_N',
       'race_O', 'race_W', 'geo_Rural', 'geo_Suburban', 'geo_Undetermined',
       'geo_Urban', 'threat_attack', 'threat_other', 'threat_undetermined'],
      dtype='object')